## This notebook use the **Mapillary API** to request images and metadata given a set of points, and storing them in locally Database. 

[Mapillary API Documentation](https://www.mapillary.com/developer/api-documentation?locale=pt_PT)

#### Import the necessary libraries

In [3]:
# Import library and some pre-installed modules
import os
import sys
from IPython.display import display, Markdown

In [4]:
# Sets the root directory of the project as the working directory
os.chdir('..')

In [5]:
# Get current working directory
os.getcwd()

'/Users/darlanmnunes/Dev/DSc_git/PhD_Thesis_Step3_OSM_Toponyms'

In [12]:
# Import and Reload the modules to ensure any changes are reflected
import importlib

import src.mapillary_api as mapillary_api
import src.mapillary_tile_downloader as mapillary_tile_downloader
import src.mapillary_metadata_enricher as mapillary_metadata_enricher

importlib.reload(mapillary_api)
importlib.reload(mapillary_tile_downloader)
importlib.reload(mapillary_metadata_enricher)

<module 'src.mapillary_metadata_enricher' from '/Users/darlanmnunes/Dev/DSc_git/PhD_Thesis_Step3_OSM_Toponyms/src/mapillary_metadata_enricher.py'>

### Mapillary Coverage Tiles - Retrieve image points

[Stack Overflow - GIS (Exporting vector tiles or saving them locally using QGIS)](https://gis.stackexchange.com/questions/458377/exporting-vector-tiles-or-saving-them-locally-using-qgis)

In [38]:
# This code cell is used to process a specific area using the Mapillary Coverage tiles do retrieve
# the points of image acquisition and save them in a GeoPackage file.
# To run this code, ensure you have the necessary Mapillary API token and the the 
# module `src.mapillary_tile_downloader` is correctly implemented.

from src.mapillary_tile_downloader import (
    ler_token_mapillary,
    processar_area_abrangente,
    salvar_resultados
)
import geopandas as gpd
from shapely.geometry import shape
import time
import json

# Configurações
TOKEN = ler_token_mapillary()
ZOOM = 14 # Nível de zoom para a área de interesse (> 14)
OUTPUT_GPKG = "results/3_mapillary_coverage/mapillary_coverage_bh.gpkg"

# Carregar área de interesse (exemplo: bbox de um GeoJSON)
with open("data/input_code1/limite_bh.geojson") as f:
    geojson = json.load(f)

# Criar geometria e obter bbox
geom = shape(geojson['features'][0]['geometry'])
bbox = list(geom.bounds)  # [minx, miny, maxx, maxy]

print(f"Bbox simplificada: {bbox}")
print(f"Zoom: {ZOOM}")

# Processar toda a área
start_time = time.time()
gdf_pontos = processar_area_abrangente(bbox, TOKEN, ZOOM)
tempo_processamento = time.time() - start_time

print(f"\nProcessamento concluído em {tempo_processamento:.2f} segundos")
print(f"Total de pontos encontrados: {len(gdf_pontos)}")

# Salvar resultados
if not gdf_pontos.empty:
    salvar_resultados(gdf_pontos, OUTPUT_GPKG)
    
    # Visualizar amostra
    print("\nAmostra dos dados:")
    print(gdf_pontos.head())
else:
    print("Nenhum ponto encontrado na área")

Bbox simplificada: [-44.06327447695697, -20.05950536981058, -43.85721816338328, -19.77674605928553]
Zoom: 14
Processando 165 tiles para a área...
  → Processados 10/165 tiles
  → Processados 20/165 tiles
  → Processados 30/165 tiles
  → Processados 40/165 tiles
  → Processados 50/165 tiles
  → Processados 60/165 tiles
  → Processados 70/165 tiles
  → Processados 80/165 tiles
  → Processados 90/165 tiles
  → Processados 100/165 tiles
  → Processados 110/165 tiles
  → Processados 120/165 tiles
  → Processados 130/165 tiles
  → Processados 140/165 tiles
  → Processados 150/165 tiles
  → Processados 160/165 tiles

Processamento concluído em 194.97 segundos
Total de pontos encontrados: 1811900
Pontos salvos em results/3_mapillary_coverage/mapillary_coverage_bh.gpkg

Amostra dos dados:
           image_id    captured_at  compass_angle       creator_id  \
0   192623852640864  1621771233793     263.109192  100125502238568   
1   528484455000560  1621771251296      59.773071  100125502238568   

In [ ]:
gdf_pontos.head()

In [16]:
import geopandas as gpd

# Importar o GeoDataFrame resultante do processamento
INPUT_GPKG = "results/3_mapillary_coverage/mapillary_coverage_bh.gpkg"
gdf = gpd.read_file(INPUT_GPKG, layer='mapillary_points')
# Exibir informações do GeoDataFrame
print(f"Total de pontos carregados: {len(gdf)}")
# Exibir as primeiras linhas do GeoDataFrame
display(gdf.head())

Total de pontos carregados: 1433598


,image_id,captured_at,compass_angle,creator_id,sequence_id,is_pano,organization_id,tile_z,tile_x,tile_y,geometry
0,2854792694743984,1621701673296,69.188881,103853795191073,6yhbktqcnm4lcoee8qh26p,False,1.805884e+15,14,6186,9120,POINT (-44.06166 -19.97378)
1,165794388771973,1621701666795,103.273102,103853795191073,6yhbktqcnm4lcoee8qh26p,False,1.805884e+15,14,6186,9120,POINT (-44.06194 -19.97374)
2,210964464051400,1621701660294,186.616745,103853795191073,byscxdw4cjpdjmk7apwv6k,False,1.805884e+15,14,6186,9120,POINT (-44.06255 -19.97369)
3,391405085453576,1621701661794,188.422760,103853795191073,byscxdw4cjpdjmk7apwv6k,False,1.805884e+15,14,6186,9120,POINT (-44.06242 -19.97371)
4,4221865561213359,1621701667295,101.169991,103853795191073,6yhbktqcnm4lcoee8qh26p,False,1.805884e+15,14,6186,9120,POINT (-44.06191 -19.97375)


In [17]:
import pandas as pd
import geopandas as gpd
from datetime import datetime, timezone

# Cria a nova coluna 'captured_date' convertendo o timestamp
gdf = gdf.copy()  # Faz uma cópia para evitar SettingWithCopyWarning
gdf.insert(
    loc=gdf.columns.get_loc('captured_at') + 1,
    column='captured_date',
    value=gdf['captured_at'].apply(
        lambda x: datetime.fromtimestamp(int(x)/1000, tz=timezone.utc).isoformat() if pd.notnull(x) else None
    )
)

# Visualize para conferir
display(gdf.head())

# Salva novamente em GPKG
gdf.to_file('results/3_mapillary_coverage/mapillary_coverage2_bh.gpkg', driver='GPKG')

,image_id,captured_at,captured_date,compass_angle,creator_id,sequence_id,is_pano,organization_id,tile_z,tile_x,tile_y,geometry
0,2854792694743984,1621701673296,2021-05-22T16:41:13.296000+00:00,69.188881,103853795191073,6yhbktqcnm4lcoee8qh26p,False,1.805884e+15,14,6186,9120,POINT (-44.06166 -19.97378)
1,165794388771973,1621701666795,2021-05-22T16:41:06.795000+00:00,103.273102,103853795191073,6yhbktqcnm4lcoee8qh26p,False,1.805884e+15,14,6186,9120,POINT (-44.06194 -19.97374)
2,210964464051400,1621701660294,2021-05-22T16:41:00.294000+00:00,186.616745,103853795191073,byscxdw4cjpdjmk7apwv6k,False,1.805884e+15,14,6186,9120,POINT (-44.06255 -19.97369)
3,391405085453576,1621701661794,2021-05-22T16:41:01.794000+00:00,188.422760,103853795191073,byscxdw4cjpdjmk7apwv6k,False,1.805884e+15,14,6186,9120,POINT (-44.06242 -19.97371)
4,4221865561213359,1621701667295,2021-05-22T16:41:07.295000+00:00,101.169991,103853795191073,6yhbktqcnm4lcoee8qh26p,False,1.805884e+15,14,6186,9120,POINT (-44.06191 -19.97375)


### Mapillary image Metadata: Enrich the points retrived from Mapillary Coverage tiles with image metadata

In [42]:
# This code cell enriches the Mapillary points retrived from Mapillary Coverage tiles with image metadata
# To apply this code, you need to have the Mapillary API token set up in your environment and 
# the module mapillary_metadata_enricher should be imported

from src.mapillary_metadata_enricher import (
    ler_token_mapillary,
    enriquecer_geodataframe,
    salvar_geodataframe_enriquecido
)
import geopandas as gpd
import time
from tqdm.notebook import tqdm
import logging

# Configurar logging detalhado
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# 1. Configuração inicial
TOKEN = ler_token_mapillary()
INPUT_GPKG = "results/3_mapillary_coverage/mapillary_coverage_bh.gpkg"
OUTPUT_GPKG = "results/3_mapillary_coverage/mapillary_coverage_bh_meta.gpkg"

# 2. Carregar dados existentes
logger.info(f"Carregando dados de {INPUT_GPKG}")
gdf = gpd.read_file(INPUT_GPKG, layer='mapillary_points')
logger.info(f"Total de pontos carregados: {len(gdf)}")
logger.info(f"Colunas originais: {list(gdf.columns)}")

# 3. Enriquecer dados
start_time = time.time()
gdf_enriched = enriquecer_geodataframe(
    gdf,
    TOKEN,
    max_workers=15,
    batch_size=10000
)
enrichment_time = time.time() - start_time

logger.info(f"\nEnriquecimento concluído em {enrichment_time/60:.2f} minutos")
logger.info(f"Colunas originais: {len(gdf.columns)}")
logger.info(f"Colunas enriquecidas: {len(gdf_enriched.columns)}")
logger.info(f"Novas colunas adicionadas: {len(gdf_enriched.columns) - len(gdf.columns)}")

# 4. Salvar resultados
salvar_geodataframe_enriquecido(gdf_enriched, OUTPUT_GPKG)

2025-06-20 10:02:21,391 - INFO - Carregando dados de results/3_mapillary_coverage/mapillary_coverage_bh.gpkg
2025-06-20 10:05:30,964 - INFO - Total de pontos carregados: 1433598
2025-06-20 10:05:30,967 - INFO - Colunas originais: ['image_id', 'captured_at', 'compass_angle', 'creator_id', 'sequence_id', 'is_pano', 'organization_id', 'tile_z', 'tile_x', 'tile_y', 'geometry']
2025-06-20 10:05:31,126 - INFO - Iniciando enriquecimento para 1433566 imagens únicas
Obtendo metadados:  23%|██▎       | 330000/1433566 [4:08:12<12:48:15, 23.94it/s]  2025-06-20 14:15:56,565 - WARNING - Erro geral para imagem 784490562453530: HTTPSConnectionPool(host='graph.mapillary.com', port=443): Read timed out. (read timeout=15)
2025-06-20 14:15:56,599 - WARNING - Erro geral para imagem 131333592394204: HTTPSConnectionPool(host='graph.mapillary.com', port=443): Read timed out. (read timeout=15)
2025-06-20 14:15:56,608 - WARNING - Erro geral para imagem 3984310591687518: HTTPSConnectionPool(host='graph.mapillary

: 

In [ ]:
# 5. Verificação detalhada se dados foram enriquecidos
logger.info("\nVerificando dados enriquecidos...")

if not gdf_enriched.empty:
    # Verificar campos originais
    original_cols = set(gdf.columns)
    preserved_cols = original_cols.intersection(set(gdf_enriched.columns))
    logger.info(f"Campos originais preservados: {len(preserved_cols)}/{len(original_cols)}")
    
    # Identificar novos campos
    new_cols = list(set(gdf_enriched.columns) - original_cols)
    logger.info(f"\nTotal de novos campos: {len(new_cols)}")
    logger.info(f"Novos campos: {new_cols}")
    
    # Verificar campos essenciais
    essential_fields = [
        'altitude', 'atomic_scale', 'camera_parameters', 'camera_type',
        'captured_at', 'compass_angle', 'computed_altitude', 'computed_compass_angle',
        'computed_geometry', 'computed_rotation', 'creator_username', 'exif_orientation',
        'geometry', 'height', 'is_pano', 'make', 'model', 'thumb_256_url',
        'thumb_1024_url', 'thumb_2048_url', 'thumb_original_url', 'merge_cc',
        'mesh_url', 'sequence', 'sfm_cluster_url', 'width', 'detections'
    ]
    
    logger.info("\nCampos essenciais presentes:")
    for field in essential_fields:
        exists = field in new_cols
        logger.info(f"{field}: {'✔' if exists else '✘'}")
    
    # Amostra de dados
    sample_cols = ['image_id'] + new_cols[:10]
    logger.info("\nAmostra de dados enriquecidos:")
    logger.info(gdf_enriched[sample_cols].head(3).to_string(index=False))

### Retrieve and download the Mapillary images

In [ ]:
# download all the images in the ROI

outfolderpath = 'sli_roi/napillary'

download_all_pictures_from_gdf(mapillary_md_gdf_filtered,outfolderpath)

In [ ]:
# Test to download in the vicinity of a Point:
buffer = 20

#p = Point(-49.27950,-25.45297)
p = Point(-42.877416, -20.755756)
buffer = radius_to_degrees(buffer, p.y)
p_bounds = p.buffer(buffer).bounds

# using the basic function to fetch mappillary data:
p_metadata = get_mapillary_images_metadata(*p_bounds,outfolderpath)
p_md_gdf = mapillary_data_to_gdf(p_metadata,outfolderpath)
download_all_pictures_from_gdf(p_md_gdf,outfolderpath)


0it [00:00, ?it/s]


In [4]:
# scan all the images in the folder
outfolderpath = './sli_roi/mapillary'
output_csv_path = './sli_roi/mapillary/image_paths.csv'
scan_images_and_save_to_csv(outfolderpath, output_csv_path)

CSV salvo em: ./sli_roi/mapillary/image_paths.csv
